In [0]:
# !pip install -U catalyst

In [3]:
!pip uninstall albumentations

Uninstalling albumentations-0.4.5:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/albumentations-0.4.5.dist-info/*
    /usr/local/lib/python3.6/dist-packages/albumentations/*
Proceed (y/n)? y
  Successfully uninstalled albumentations-0.4.5


In [4]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-efnclhma
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-efnclhma
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=65038 sha256=c4d9c8f21cdbbe35afda7f262d55b314a59d9eb19d9a72db4bfce2ffc3b40e97
  Stored in directory: /tmp/pip-ephem-wheel-cache-ma0zkqjs/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
Successfully built albumentations


In [67]:
import numpy as np
import pandas as pd
import os, random
import cv2
import glob
import collections
import time
import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu
from albumentations import pytorch as AT
from albumentations.pytorch import ToTensorV2

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

#Jupyterでインライン表示するための宣言
%matplotlib inline #追加
# import catalyst
# from catalyst.dl import SupervisedRunner
# from catalyst.dl.callbacks import AccuracyCallback


UsageError: unrecognized arguments: #追加


In [0]:
name_labels = {'EmperorPenguin': 1, 'RockhopperPenguin': 2, 'GalapagosPenguin': 3, 'KingPenguin':4,
         'White-flipperedPenguin':5,'HumboldtPenguin': 6 ,'MagellanicPenguin': 7,'AfricanPenguin': 8,'LittlePenguin': 9,
         'ChinstrapPenguin': 10,'FiordlandPenguin':11,'Erect-crestedPenguin':12,'SnaresIslandsPenguin':13,'RoyalPenguin':14,
         'MacaroniPenguin':15,'AdeliePenguin':16,'GentooPenguin':17,'Yellow-eyedPenguin':18}

In [0]:
class PRDataset(Dataset):
    def __init__(self, datatype: str = 'train', transforms=None):
        self.image_paths = []
        self.labels = []
        self.transforms = transforms 
        self.datatype = datatype
        path = './images'
        # 拡張子がjpegのみ取得
        if datatype == 'train':
          file_paths = glob.glob('./images/*' + '/*.jpg')
        elif datatype == 'valid':
          file_paths = glob.glob('./val/*.jpg')
        else:
          file_paths = glob.glob('./test/*.jpg')
        for path in file_paths:
          self.image_paths.append(path)
          if datatype != 'test':
            label = path.split('/')[-1].split('_')[0]
            self.labels.append(name_labels[label])

        self.num_samples = len(self.image_paths)
        print(f"[Detection Dataset] size: {self.num_samples}")

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        augmented = self.transforms(image=image)
        image = augmented['image']
        if self.datatype != 'test':
          label = self.labels[index]
          return image, label
        else:
          # Noneだとダメらしい
          return image, "None"   


    def __len__(self):
        return self.num_samples

In [0]:
img_height = 256
img_width = 256

# train用のデータ拡張
data_transforms = albu.Compose([
    albu.Resize(img_height, img_width),
    albu.HorizontalFlip(p=0.5),
    albu.Normalize(),
    ToTensorV2()
])

# test用のデータ拡張
data_transforms_test = albu.Compose([
    albu.Resize(img_height, img_width),
    albu.Normalize(),
    ToTensorV2()
    ])

In [9]:
train_dataset = PRDataset(datatype='train', transforms=data_transforms)
valid_dataset = PRDataset(datatype='valid', transforms=data_transforms_test)

num_workers = 0
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

[Detection Dataset] size: 2218
[Detection Dataset] size: 89


In [0]:
def train(model, data_loader, criterion, optimizer, device, grad_acc=1):
    model.train()
    # zero the parameter gradients
    optimizer.zero_grad()
    total_loss = 0.
    for i, (inputs, labels) in enumerate(data_loader):
        inputs = inputs.to(device)
        labels=torch.from_numpy(np.array(labels))
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        # Gradient accumulation
        if (i % grad_acc) == 0:
            optimizer.step()
            optimizer.zero_grad()
        total_loss += loss.item()
    total_loss /= len(data_loader)
    metrics = {'train_loss': total_loss}
    return metrics
def eval(model, data_loader, criterion, device):
    model.eval()
    num_correct = 0.
    with torch.no_grad():
        total_loss = 0.
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            num_correct += torch.sum(preds == labels.data)
        total_loss /= len(data_loader)
        num_correct /= len(data_loader.dataset)
        metrics = {'valid_loss': total_loss, 'val_acc': num_correct}
    return metrics


In [0]:
import torchvision.models as models
model = models.resnext50_32x4d(pretrained=True)
# model = models.resnext50_32x4d("output.pth")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
epochs = 5
num_class = 18
output_path = './output/catalyst'
loaders = {"train": train_loader, "valid": valid_loader}

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
epochs = 5
num_class = 18

epochs = 7
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model
criterion = criterion
# Model を multi-gpu したり、FP16 対応したりする
model = model.to(device)
print('Train start !')
for epoch in range(epochs):
    print(f'epoch {epoch} start !')
    metrics_train = train(model, train_loader, criterion, optimizer, device)
    metrics_eval = eval(model, valid_loader, criterion, device)
    scheduler.step()
    # Logger 周りの処理
    # print するためのごちゃごちゃした処理
    print(f'epoch: {epoch} ', metrics_train, metrics_eval)

Train start !
epoch 0 start !
epoch: 0  {'train_loss': 0.7354396602937153} {'valid_loss': 2.068526824315389, 'val_acc': tensor(0.5281, device='cuda:0')}
epoch 1 start !
epoch: 1  {'train_loss': 0.5696406903011458} {'valid_loss': 2.4503137270609536, 'val_acc': tensor(0.5056, device='cuda:0')}
epoch 2 start !
epoch: 2  {'train_loss': 0.4333643302321434} {'valid_loss': 2.071386913458506, 'val_acc': tensor(0.5506, device='cuda:0')}
epoch 3 start !
epoch: 3  {'train_loss': 0.3016752179179873} {'valid_loss': 1.6000480651855469, 'val_acc': tensor(0.6180, device='cuda:0')}
epoch 4 start !
epoch: 4  {'train_loss': 0.33400224191801886} {'valid_loss': 2.222716808319092, 'val_acc': tensor(0.5506, device='cuda:0')}
epoch 5 start !
epoch: 5  {'train_loss': 0.1846762572016035} {'valid_loss': 1.8656799793243408, 'val_acc': tensor(0.5955, device='cuda:0')}
epoch 6 start !
epoch: 6  {'train_loss': 0.11048330409186227} {'valid_loss': 1.8652525742848713, 'val_acc': tensor(0.6966, device='cuda:0')}


In [0]:
torch.save(model.state_dict(), './output_7.pth')

## testデータで検証する！！！！！

In [63]:
test_dataset = PRDataset(datatype='test', transforms=data_transforms_test)
num_workers = 0
batch_size = 1
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

[Detection Dataset] size: 98


In [0]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [0]:
def predict(model, data_loader, device):
    predict_list = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
          #貼り付け
            inputs = inputs.to(device)
            # labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predict_list.append(preds[0])
    return predict_list

In [0]:
label = predict(model, test_loader, device)

In [0]:
def get_key_from_value(d, val):
    keys = [k for k, v in d.items() if v == val]
    if keys:
        return keys[0]
    return None

In [75]:
test_paths = glob.glob('./test/*.jpg')
for index, path in enumerate(test_paths):
  im = Image.open(path)
  #画像をarrayに変換
  im_list = np.asarray(im)
  plt.title(get_key_from_value(name_labels, label[index]))
  #貼り付け
  plt.imshow(im_list)
  #表示
  plt.show()
    

Output hidden; open in https://colab.research.google.com to view.